In [ ]:
#  get names of .tif files in a folder in google drive to a list

import os
from google.colab import drive
import re

# Mount your Google Drive
drive.mount('/content/drive')

# Get the path to the folder containing the .tif files
folder_path = '/content/drive/My Drive/ColabSamples'

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Filter the list to only include .tif files
extensions = ('.tiff', '.tif')
tif_files = [file_name for file_name in files if file_name.lower().endswith(extensions)]
#tif_files = os.listdir(folder_path)
# Print the list of .tif files
print(files)
print('end of files')
print(tif_files)


# Filter out from the list of tif files
# as the keyword "ref" in the filename (reference files) and others (filtered files)
#reference_files = [f for f in tif_files if re.match(r'ref', f)]
filtered_files = []
reference_files = []
for filename in tif_files:
  if "ref"  in filename:
    reference_files.append(filename)
  else  :
    filtered_files.append(filename)

# Print the filtered list of .tif files
print('observation images', filtered_files)
print("end of observation images")
print('reference images:', reference_files)

#done

Mounted at /content/drive
['gözlem12.05.22_1.TIF', 'gözlem12.05.22_ref 1.TIF', 'gözlem12.05.22_ref 2.TIF', 'gözlem12.05.22_8.TIF', 'gözlem12.05.22_7.TIF', 'gözlem12.05.22_6.TIF', 'gözlem12.05.22_5.TIF', 'gözlem12.05.22_4.TIF', 'gözlem12.05.22_3.TIF', 'gözlem12.05.22_2.TIF', 'BohemanRhapsody Melody Note Book.pdf']
end of files
['gözlem12.05.22_1.TIF', 'gözlem12.05.22_ref 1.TIF', 'gözlem12.05.22_ref 2.TIF', 'gözlem12.05.22_8.TIF', 'gözlem12.05.22_7.TIF', 'gözlem12.05.22_6.TIF', 'gözlem12.05.22_5.TIF', 'gözlem12.05.22_4.TIF', 'gözlem12.05.22_3.TIF', 'gözlem12.05.22_2.TIF']
observation images ['gözlem12.05.22_1.TIF', 'gözlem12.05.22_8.TIF', 'gözlem12.05.22_7.TIF', 'gözlem12.05.22_6.TIF', 'gözlem12.05.22_5.TIF', 'gözlem12.05.22_4.TIF', 'gözlem12.05.22_3.TIF', 'gözlem12.05.22_2.TIF']
end of observation images
reference images: ['gözlem12.05.22_ref 1.TIF', 'gözlem12.05.22_ref 2.TIF']


In [ ]:
# from PIL import Image
from PIL.ExifTags import TAGS
import pathlib
import csv

import glob
from PIL import Image
import pandas as pd


index_i_max = len(filtered_files)
index_i = 0 # counter in list of filtered files
ref_switch = 0 # switch between reference files

#img = Image.open('test_tif/sample.tif')
image = Image.open(os.path.join(folder_path, filtered_files[index_i]))

# extract other basic metadata
info_dict = {
    "Filename": image.filename.split('/')[1],
    "Image Size": image.size,
    "Image Height": image.height,
    "Image Width": image.width,
    "Image Format": image.format,
    "Image Mode": image.mode,
    "Image is Animated": getattr(image, "is_animated", False),
    "Frames in Image": getattr(image, "n_frames", 1),
}

print(info_dict)

#info.append(info_dict)

#done

{'Filename': 'content', 'Image Size': (2048, 2048), 'Image Height': 2048, 'Image Width': 2048, 'Image Format': 'TIFF', 'Image Mode': 'I;16', 'Image is Animated': False, 'Frames in Image': 1}


In [ ]:
#  in filtered_files , first element in list matching document if it is a grayscale tif file, read it to matrix

import tifffile as tiff
#import os

print(len(files),", ",len(tif_files))
print(len(filtered_files))
print(len(reference_files))

if not os.path.isdir(folder_path):
    print("The 'folder_path' variable is invalid.")
    # Investigate why the folder path is invalid (e.g., check if the folder exists)

# Get the path to the first filtered file
filtered_files_path = os.path.join(folder_path, filtered_files[index_i])
reference_files_path = os.path.join(folder_path, reference_files[ref_switch])

# Read the grayscale tif file to a matrix
image_matrix = tiff.imread(filtered_files_path)
reference_matrix = tiff.imread(reference_files_path)
# Print the shape of the matrix
print(image_matrix.shape)
print(reference_matrix.shape)

#done

16 ,  10
8
2
(2048, 2048)
(2048, 2048)


In [ ]:
#  create new difference_matrix, size [n,n] ; compare each [n,n] value from "image_matrix" and "reference_matrix" and, if difference bigger than 20 write to difference_matrix [n,n] , there n=2048

import numpy as np
n=2047
m=2047

#print (reference_matrix[n, m])
#print (image_matrix[n, m])


#compare reference and image, pixel by pixel
#if difference little as var=10 mark position in difference matrix with value 120
#if bigger than 10 get image value and position, so that we got 'where cells are'

#16 bit grayscale has 2^16 = 65536 tone and tif doesnt support it with alpha channel (transparency)
#for now reduce it to 8 bit with /256

difference_matrix = np.zeros((n, m, 4))
for i in range(n):
    for j in range(m):
        if ((reference_matrix[i, j]/256) - (image_matrix[i, j])/256) > 10:
            difference_matrix[i, j] = (image_matrix[i, j]/256)
            difference_matrix[i, j, 3] = 120

#temp vers 1    works

In [ ]:
# Create a 100x100 grayscale image with transparent background
#import numpy as np
#import tifffile as tiff     ---çalışıyor--- 32bit RGB

#n=100
#m=100
#new_save_name = 'deneme_149_1a'
new_save_name = 'deneme_149_' + str(index_i) + '_1a'

new_image = np.zeros((n, m, 4), dtype=np.uint8)
new_image[:, :, 3] = 0  # Set alpha channel to 0 (transparent) ; (0,1,2)=(RGB) ; (3)=alpha

# Save the image as a grayscale tif file
#tiff.imsave('new_image.tif', new_image, dtype='uint8')
new_file_path = os.path.join(folder_path, new_save_name)
tiff.imsave(new_file_path, difference_matrix)

#temp vers 1 a works

<ipython-input-46-51322427c831>:16: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(new_file_path, difference_matrix)


In [ ]:
# for repetitive process
for index_i in range(index_i_max):
    # Get the path to the index_i'th filtered file
    filtered_files_path = os.path.join(folder_path, filtered_files[index_i])
    # Read the grayscale tif file to a matrix
    image_matrix = tiff.imread(filtered_files_path)
    # Print the shape of the matrix
    print(image_matrix.shape)
    difference_matrix = np.zeros((n, m, 4))
    for i in range(n):
        for j in range(m):
            if ((reference_matrix[i, j]/256) - (image_matrix[i, j])/256) > 10:
                difference_matrix[i, j] = (image_matrix[i, j]/256)
                difference_matrix[i, j, 3] = 120
    new_save_name = 'deneme_149_' + str(index_i) + '_1a'
    new_file_path = os.path.join(folder_path, new_save_name)
    tiff.imsave(new_file_path, difference_matrix)









(2048, 2048)


<ipython-input-50-333fdbebfc4e>:17: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(new_file_path, difference_matrix)


(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)
(2048, 2048)


below there only test purpose

In [ ]:
# !pip install tifffile==2024.5.22

n=100
m=100
#, dtyupe=np.uint16)
grayscale_data = np.random.randint(0, 65535, (n,m), np.dtype('uint16'))
alpha_data = np.full((n, m), 255, dtype=np.uint8) # full opaque

# combine grayscale and alpha chanel
combined_data = np.stack([grayscale_data, alpha_data], axis=-1)

#save tiff file with combined_data
filename = 'new_image_aaa1.tif'
filepath = os.path.join(folder_path, filename)
tiff.imsave(filepath, combined_data)

#temp vers 1 b -not working

<ipython-input-9-e7c67a4719da>:15: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(filepath, combined_data)


In [ ]:
# create a new tif file with part of that matrix row: 20 to 130, column: 40 to 160 and 10 transparent on each side

import numpy as np
# Create a new matrix with the specified dimensions
new_image_matrix = np.zeros((110, 120, 4), dtype=np.uint8)

# Copy the relevant portion of the original matrix to the new matrix
new_image_matrix[10:-10, 10:-10, :] = image_matrix[20:130, 40:160, :]

# Save the new matrix to a new tif file
new_file_path = os.path.join(folder_path, 'new_image.tif')
tiff.imsave(new_file_path, new_image_matrix)

# temp vers 1 c -not working

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
# prompt: create new difference_matrix, size [n,n] ; compare each [n,n] value from "image_matrix" and "reference_matrix" and, if difference bigger than 20 write to difference_matrix [n,n] , there n=2048

import numpy as np
#n=2047
#m=2047

n=1500
m=700

#print (reference_matrix[n, m])
#print (image_matrix[n, m])


#compare reference and image, pixel by pixel
#if difference little as var=10 mark position in difference matrix with value 120
#if bigger than 10 get image value and position, so that we got 'where cells are'

#16 bit grayscale has 2^16 = 65536 tone and tif doesnt support it with alpha channel (transparency)
#for now reduce it to 8 bit with /256

alpha_data = np.full((n, m), 255, dtype=np.uint8) # full opaque

difference_matrix = np.zeros((n, m, 1)) #4
for i in range(n):
    for j in range(m):
        if ((reference_matrix[i, j]/256) - (image_matrix[i, j])/256) > 10:
            difference_matrix[i, j] = (image_matrix[i, j]/256)
        alpha_data[i, j] = 100
            #difference_matrix[i, j, 1] = 27 #3 difference_matrix[i, j, 3] = 120 #

        alarm = 1

#temp vers 2

In [ ]:
# Create a 100x100 grayscale image with transparent background
#import numpy as np
#import tifffile as tiff

print (alarm)

#n=100
#m=100
new_save_name = 'deneme_151_2a'

# Save the image as a grayscale tif file
#tiff.imsave('new_image.tif', new_image, dtype='uint8')
new_file_path = os.path.join(folder_path, new_save_name)
tiff.imsave(new_file_path, difference_matrix)


new_image = np.zeros((n, m, 2), dtype=np.uint8)
new_image[:, :, 1] = 253  # Set alpha channel to 0 (transparent) ; (0,1,2)=(RGB) ; (3)=alpha


# Stack grayscale and alpha channels
image_data = np.stack([difference_matrix, alpha_data], axis=-1)


# temp vers 2 a

1


<ipython-input-36-81589cfe3bf5>:14: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(new_file_path, difference_matrix)


ValueError: all input arrays must have the same shape